# Data Reducing Notebook

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import time
from datetime import datetime

consumerKey = 'hnuzMJ1QGE3TcjlYiXohfA6vkGb55OT9xKtTCcd31L4'

### Get Summary Data

The goal is to obtain the floating population data (point type) from [Tokyo Public Transportation Open Data Challenge](https://ckan-tokyochallenge.odpt.org/dataset/o_fpd_point-agoop). This data however is very large, and we undesirable to store in memory. Thus, we need to reduce the data along the way and to produce the dataset we want. First lets download the summary dataset.

In [ ]:
summary_url = 'https://api-tokyochallenge.odpt.org/api/v4/files/Agoop/data/Summary.csv?acl:consumerKey={}'
df_summary = pd.read_csv(summary_url.format(consumerKey))

In [ ]:
df_summary.date = [datetime.strptime(_d, '%Y/%m/%d') for _d in df_summary.date]
plt.figure(figsize=(16,8), dpi=80)
plt.subplot(121)
sns.lineplot(x='date', y='dailyid_count', 
             data=df_summary,
             estimator=None
            )
plt.title('Daily Id count')
plt.subplot(122)
sns.lineplot(x='date', y='log_count', 
             data=df_summary,
             estimator=None
            )
plt.title('Daily log count')
plt.show()

The range of the data is from 2018/10/01 to 2019/09/30 (365 days). Our variable of interset is the dailyid_count. Our objective is to gather 37 days worth of data.

From the plot we can see that the data from June to August seems relatively stable. They don't decrease or increase suddenly. Thus we will gather data from 2019/06/01 to 2019/7/7.

In [ ]:
citymaster_url = "https://api-tokyochallenge.odpt.org/api/v4/files/Agoop/data/prefcode_citycode_master_UTF-8.csv?acl:consumerKey={}"
df_citycode = pd.read_csv(citymaster_url.format(consumerKey))

tokyo_32_dict = dict(df_citycode[df_citycode.prefname=='東京都'].loc[:,['cityname', 'citycode']].values[:32])

In [ ]:
url = 'https://api-tokyochallenge.odpt.org/api/v4/files/Agoop/data/PDP_20190721_10.csv?acl:consumerKey='
df_test = pd.read_csv(url + consumerKey)

df_test.head()

First we want to subset the dataframe to the cities we are interested in so as to reduce the processing load in the aggregation process. The city codes we are interested in is contained within values of the `tokyo_32_dict`.
We can do this smartly by setting the `citycode` as the index and use the `.loc` method to subset the data. To de-index `citycode` we simply call the `reset_index()` method.

Next, we want to find the the count of unique `dailyid` for each `citycode`. We can do this grouping by the `citycode` and then by using `nunique` method on the `dailyid` column. Also, since we only need the id counts, we will remove all other columns by subsetting only the `dailyid` column.

finally we can encapsulate this whole process into a function and pass it to the preprocessing chain using the `pipe` method.

In [ ]:
def count_id_by_city(df):
    """
    Counts the unique daily id within each city
    """
    return (df.set_index(['citycode'])
            .loc[tokyo_32_dict.values(),['dailyid']]
            .reset_index()
            .groupby(['citycode'])
            .nunique(['dailyid'])
            .drop(['citycode'], axis=1)
            .reset_index()
           )

We also want to add the date and time in our dataset in order to plot the time series. However this information is contained when we download the dataset from the database. Ideally, we want to automatically generate the datetime strings for a certain range. 

We want to take strings to set our beginning and set how many days worth or data we want to obtain. We can use `data_range` method in python to obtain a range of dates(we can also set the frequency) and then use the `strftime` function in `datetime` library to convert them into the string we want. 

In [ ]:
def append_timestamp(df, date):
    df['datetime']  = date
    return df

Finally, we can use the pandas `pipe` the create a process chain. This makes the code much more readble. 

In [ ]:
(df_test
 .pipe(count_id_by_city)
 .pipe(append_timestamp, '2019-07-21')
 .head()
)

In [ ]:
def get_data(start: str, days: int):
    """
    start: starting date in 'yyyy-mm-dd' format
    days: Number of days of data we want to obtain 
    """
    url = 'https://api-tokyochallenge.odpt.org/api/v4/files/Agoop/data/PDP_{}.csv?acl:consumerKey={}'
    consumerKey = 'hnuzMJ1QGE3TcjlYiXohfA6vkGb55OT9xKtTCcd31L4'
    
    periods = days * 24
    df_list = []
    for date in pd.date_range(start=start, periods=periods, freq='H').tolist():
        date_str = datetime.strftime(date, "%Y%m%d_%H")
        try:
            df = (pd.read_csv(url.format(date_str, consumerKey))
                  .pipe(count_id_by_city)
                  .pipe(append_timestamp, date)
                 )
            df_list.append(df)
            
            print('getting data from: {}'.format(date))
            
            time.sleep(1)
        except:
            continue
            
    return pd.concat(df_list)

In [ ]:
df = get_data('2019-06-01', 10)